<a href="https://colab.research.google.com/github/blackrosedragon2/Text-Summarization/blob/master/TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tqdm
!pip install pyspellchecker
!pip install torchsummaryx

In [0]:
#Reading data onto zipfile object ZF
from google.colab import drive
import os 
import zipfile
drive.mount('/content/gdrive')
root_path = '/content/gdrive/My Drive/data/'  
zf=zipfile.ZipFile(root_path+"cnn_stories.zip")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import re
from spellchecker import SpellChecker
class preprocess:
    #Accepts a string and performs string operations on it
    def seperate(self,para):#1
        # find @highlight, add length and split story and summary, then decode to utf-8 format, use split tokenizer and seperate the 4 lines , split the \n parts and return the list
        story=para[:para.find('@highlight')]
        summary=[i.splitlines()[2] for i in (para[para.find('@highlight')+len('@highlight'):]).split('@highlight')]
        return story,summary    
        #returns (string,list)
    
    def spell_check(self,para):#2
        #corrects spellings(may not be used for us) + takes much longer to correct spellings (5+ minutes)
        spell = SpellChecker()
        para=para.split(' ')
        correctpara=[]
        for word in para:
            correctpara.append(spell.correction(word))
        return ' '.join(correctpara)

    def remove_apostrophes(self,para):#3
        return re.sub("(?<=[a-z])'(?=[a-z])", "", para)

    def remove_parentheses(self,para):#4
        #removes anything inside parentheses
        return re.sub(r'\([^)]*\)', '', para)

    def remove_sp(self,para):#5
        #removes special symbols
        return re.sub('[^A-Za-z0-9]+', ' ', para)   #note: add exception for words like U.N. 

    def remove_short(self,para):#6
        #removes words that are <3 in length
        para=para.split(' ')
        longpara=[]
        for word in para:
            if len(word) >= 3:
                longpara.append(word)
        return ' '.join(longpara)

    def add_tags(self,para):#7
        #add start and end tags 
        para="<start>"+para+"<end>"
        return para

    def apply_all(self,para):#8
        #applies 1,3,4,5,6 functions, may need reordering 
        p=preprocess()

        story,summary=p.seperate(para.decode("utf-8"))

        #story=p.spell_check(story)
        #summary=p.spell_check(summary)

        story=p.remove_apostrophes(story)
        summary=[p.remove_apostrophes(i) for i in summary] 

        story=p.remove_parentheses(story)
        summary=[p.remove_parentheses(i) for i in summary]

        story=p.remove_sp(story)
        summary=[p.remove_sp(i) for i in summary]

        story=p.remove_short(story)
        summary=[p.remove_short(i) for i in summary]

        return story,summary



In [0]:
#Read content of all files onto x (takes approximatly 13 minutes for both modes)
from tqdm import tqdm

def get_data(preproc=0):
    i=1
    if preproc==1:
        stories=[]
        story=[]
        summaries=[]
        summary=[]
        p=preprocess()
        with tqdm(total=92600) as pbar:
            while(True):
                #Reads zip file until eof exception occurs
                i=i+1
                try:
                    story,summary=p.apply_all(zf.open(zf.namelist()[i]).read())
                    stories.append(story)
                    summaries.append(summary)
                except Exception as E:
                    break
                pbar.update(1)
        return stories,summaries
    else:
        x=[]
        with tqdm(total=92600) as pbar:
            while(True):
                #Reads zip file until eof exception occurs
                i=i+1
                try:
                    x.append(zf.open(zf.namelist()[i]).read())
                except Exception as E:
                    break
                pbar.update(1)
        return x

_,summa=get_data(1) 
for j,i in enumerate(summa):
    print(i)
    if j>10:
        break

100%|█████████▉| 92579/92600 [09:25<00:00, 163.81it/s]

['Syrian official Obama climbed the top the tree doesnt know how get down', 'Obama sends letter the heads the House and Senate', 'Obama seek congressional approval military action against Syria', 'Aim determine whether were used not whom says spokesman']
['Usain Bolt wins third gold world championship', 'Anchors Jamaica 4x100m relay victory', 'Eighth gold the championships for Bolt', 'Jamaica double womens 4x100m relay']
['The employee agencys Kansas City office among hundreds virtual workers', 'The employees travel and from the mainland last year cost more than 000', 'The telecommuting program like all GSA practices under review']
['NEW Canadian doctor says she was part team examining Harry Burkhart 2010', 'NEW Diagnosis autism severe anxiety post traumatic stress disorder and depression', 'Burkhart also suspected German arson probe officials say', 'Prosecutors believe the German national set string fires Los Angeles']
['Another arrest made gang rape outside California school', 'Inves

In [0]:
%xmode Plain
import torch.nn as nn
import torch
import numpy as np
from torchsummaryX import summary
device = torch.device("cuda")

class GRU_Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop=0.2):
        super(GRU_Encoder, self).__init__()
        #GRU with dropout,fc layer, relu layer
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers*2 # bidirectional

        self.embedding = nn.Embedding(input_dim, hidden_dim)
        self.gru = nn.GRU(hidden_dim, hidden_dim, n_layers, batch_first=True, dropout=drop, bidirectional =True) 
        #batch first will create dims such that dimensions=(batch,dim1,dim2...)

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data      
         #grabs any parameter of the model and uses it to instantiate (through .data.new) a new tensor on the same device
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        #ints a hidden vector shape( n_layer,batch_size,hidden_dim) to zero
        return hidden

    def forward(self, x, h=0):# x dims=()
        if h==0:
            h=self.init_hidden(32)

        embedded = self.embedding(x).view(x.shape[0],-1,self.hidden_dim)
        out=embedded
        out, h = self.gru(out, h)  
        print("X {},OUTPUT {},HIDDEN {}".format(x.shape,out.shape,h.shape))

        return out, h  #(output of encoder(give to-> attention mechanism ), next hidden state)

#input trensor dims must be (i/p dim,timesteps,batchsize)
gruEncoder_model=GRU_Encoder(input_dim=100, hidden_dim=128, output_dim=20, n_layers=10)
inputs = torch.zeros((32,20, 100), dtype=torch.long) # (batch_size, outputdim, inputdim)
inputs=inputs.cuda()
gruEncoder_model.cuda()
summary(gruEncoder_model,inputs)



Exception reporting mode: Plain
X torch.Size([32, 20, 100]),OUTPUT torch.Size([32, 2000, 256]),HIDDEN torch.Size([20, 32, 128])
            Kernel Shape        Output Shape   Params  Mult-Adds
Layer                                                           
0_embedding   [128, 100]  [32, 20, 100, 128]    12800      12800
1_gru                  -     [32, 2000, 256]  2866176    2850816
----------------------------------------------------------------
                       Totals
Total params          2878976
Trainable params      2878976
Non-trainable params        0
Mult-Adds             2863616


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_embedding,"[128, 100]","[32, 20, 100, 128]",12800,12800
1_gru,-,"[32, 2000, 256]",2866176,2850816


In [0]:
'''class GRU_AttnDecoder(nn.Module):
    def __init__(self, hidden_dim, output_dim,n_layers, dropout = 0.2, max_length = 15):
        super(GRU_AttnDecoder, self).__init__()
        self.n_layers=n_layers
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.max_length = max_length
        
        self.embedding = nn.Embedding(self.output_dim, self.hidden_dim)
        self.attn = nn.Linear(self.hidden_dim * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_dim * 2, self.hidden_dim)
        self.gru = nn.GRU(self.hidden_dim, self.hidden_dim,n_layers, batch_first=True, dropout=dropout, bidirectional =False)
        self.out = nn.Linear(self.hidden_dim, self.output_dim)
        
    def forward(self, input, hidden=1):
        if hidden==0:
            hidden=self.initHidden()

        embedded = self.embedding(input).view(1,1,-1)
        
        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]),1)),dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0)) 
        #torch.bmm performs batch matrix multiplication
        
        output = torch.cat((embedded[0], attn_applied[0]),1)
        output = self.attn_combine(output).unsqueeze(0)
        
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        
        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights
    
    def initHidden(self):
        return torch.zeros(1,1,self.hidden_dim, device=device)'''

class GRU_AttentionDecoder(nn.Module):
    def __init__(self, hidden_size, output_size,n_layers,dropout=0.2):
        super(GRU_AttentionDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers= n_layers

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size,n_layers,batch_first=True, dropout=dropout, bidirectional =False) 
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax()

    def forward(self, input, hidden=1):
        if hidden==1:
            hidden=self.initHidden(32)
        output = self.embedding(input).view(input.shape[0],-1,self.hidden_size)
        print("INPUT {},OUTPUT {},HIDDEN {}".format(input.shape,output.shape,hidden.shape))

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))

        return output, hidden

    def initHidden(self, batch_size):
        weight = next(self.parameters()).data      
        hidden = weight.new(self.n_layers, batch_size, self.hidden_size).zero_().to(device)
        return hidden


decoder_model=GRU_AttentionDecoder( hidden_size=128, output_size=20,n_layers=10)
inputs = torch.zeros((32,20,128), dtype=torch.long) # (batch_size, outputdim, inputdim)
inputs=inputs.cuda()
decoder_model.cuda()
summary(decoder_model,inputs)

INPUT torch.Size([32, 20, 128]),OUTPUT torch.Size([32, 2560, 128]),HIDDEN torch.Size([10, 32, 128])
            Kernel Shape        Output Shape   Params Mult-Adds
Layer                                                          
0_embedding    [128, 20]  [32, 20, 128, 128]    2.56k     2.56k
1_gru                  -     [32, 2560, 128]  990.72k   983.04k
2_out          [128, 20]          [2560, 20]    2.58k     2.56k
3_softmax              -          [2560, 20]        -         -
-----------------------------------------------------------------
                       Totals
Total params          995.86k
Trainable params      995.86k
Non-trainable params      0.0
Mult-Adds             988.16k


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_embedding,"[128, 20]","[32, 20, 128, 128]",2560.0,2560.0
1_gru,-,"[32, 2560, 128]",990720.0,983040.0
2_out,"[128, 20]","[2560, 20]",2580.0,2560.0
3_softmax,-,"[2560, 20]",NaN,NaN


In [0]:
32*20*128

81920